In [21]:
import os

import schematics as scm
import schematics.types as scmt

import pandas as pd
import numpy as np
import tensorboard.notebook as tbnb
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfl
import tensorflow.keras.preprocessing as tfpp
import tensorflow.data as tfd

%cd ~/flatiron/python
import flatiron.core.tools as fict
import flatiron.core.dataset as ficd
import flatiron.core.logging as ficl
import flatiron.models.unet as fimu
import flatiron.core.pipeline as ficp

tf.get_logger().setLevel('ERROR')

/home/ubuntu/flatiron/python


In [37]:
with open('../resources/slack-url.txt') as f:
    url = f.read().strip('\n')
config = f'''
model:
    input_width: 208
    input_height: 208
    input_channels: 3
    classes: 1
    filters: 64
    layers: 9
    activation: leaky_relu
    batch_norm: true
    output_activation: sigmoid
    # kernel_initializer:
    attention_gates: true
    # attention_activation_1:
    # attention_activation_2:
    # attention_kernel_size:
    # attention_strides:
    # attention_padding:
    # attention_kernel_initializer:
dataset:
    source: /mnt/storage/projects/unet001/dset001/p-unet001_s-dset001_d-glom_v001
    split_index: -1
    # load_limit: 1000
optimizer:
    name: sgd
    learning_rate: 0.015
    momentum: 0.99
compile:
    loss: jaccard_loss
    metrics:
        - jaccard
        - dice
callbacks:
    project: unet001
    root: /mnt/storage/projects
fit:
    batch_size: 32
    validation_split: 0.1
logger:
    timezone: 'America/Los_Angeles'
    slack_url: '{url}'
    slack_channel: dev
    slack_methods:
        - fit
'''
pipe = fimu.UNetPipeline \
    .from_string(config) \
    .load() \
    .xy_split()

# pipe = fimu.UNetPipeline \
#     .from_string(config) \
#     .load() \
#     .xy_split() \
#     .unload() \
#     .build() \
#     .compile() \
#     .fit()

# pipe = fimu.UNetPipeline \
#     .from_string(config) \
#     .run()

Loading Dataset Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

In [36]:
def preprocess(x, axis=-1, index=-1):
    data = tfk.Sequential([
        tfl.RandomFlip(seed=42),
    ])(x)
    return data

dset = tfd.Dataset.from_tensor_slices([pipe.dataset.data])
dset.batch(32).map(lambda x: preprocess(x))